<a href="https://colab.research.google.com/github/ankesh86/LLMProjects/blob/main/Structuring_LangChain_LLaMa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Schemas with LLaMa
## Goal get a python object back from LLaMa
1. Define a pydantic schema
2. Define a langchain series of prompts
3. Get a python object in return

In [1]:
!pip install langchain-groq langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [7]:
from google.colab import userdata
model = ChatGroq(temperature=0.1, groq_api_key=userdata.get('GROQ_API_KEY'), model_name="llama3-70b-8192")

In [8]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [12]:
# Here's another example, but with a compound typed field.
class Animal(BaseModel):
    name: str = Field(description="type of animal")
    likes_to_eat: List[str] = Field(description="list of foods it likes")


animal_query = "Generate what foods a random animal likes to eat. Make it a sea animal."

parser = PydanticOutputParser(pydantic_object=Animal)

prompt = PromptTemplate(
    template="Generate only the JSON file.  \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [13]:
chain = prompt | model | parser

animal = chain.invoke({"query": animal_query})

In [14]:
animal.name

'Octopus'

In [15]:
animal.likes_to_eat

['Shrimp', 'Crab', 'Fish']